## 0. Install Dependecies

In [27]:
pip install torch

**Set device to GPU if is available otherwise set device as cpu**

In [28]:
import torch
# Check if GPU is available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


**Import libraries**

In [29]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.base import BaseEstimator, RegressorMixin

##1. Dataset##

- The Datatset used in this project was generated using the Mujoco simulator with three different configurations:
- 2D (2 joints)
- 2D (3 joints)
- 3D (5 joints)

The format of the data is in CSV format, including information about Joint angles, fingertip position, and orientation.


1.1. Visualise data from the simulator

In [30]:
!head -5 logfiler2.csv

j0;j1;cos(j0);cos(j1);sin(j0);sin(j1);ft_x;ft_y;ft_qw;ft_qz
 0.055; -0.012;  0.998;  1.000;  0.055; -0.012;  0.210;  0.010;  1.000;  0.021
 0.076; -0.017;  0.997;  1.000;  0.076; -0.017;  0.210;  0.014;  1.000;  0.030
 0.148; -0.011;  0.989;  1.000;  0.147; -0.011;  0.208;  0.030;  0.998;  0.068
 0.214;  0.048;  0.977;  0.999;  0.212;  0.048;  0.204;  0.050;  0.991;  0.131


In [31]:
!head -5 logfiler3.csv

head: cannot open 'logfiler3.csv' for reading: No such file or directory


In [32]:
!head -5 logfiler5.csv

head: cannot open 'logfiler5.csv' for reading: No such file or directory


1.2. Preprocess the data


2R Robot

In [90]:
# Load dataset
data = pd.read_csv('logfiler2.csv', delimiter=';')

# Preprocessing: Extract inputs (joint angles and their trigonometric functions) and outputs (fingertip positions and quaternions)
X = data[['j0', 'j1']].values
y = data[['ft_x', 'ft_y']].values

# Normalize input features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

- Split the data into training and testing sets

In [91]:
# Split data into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

##2. Train Forward Kinematics Models##



### 2.1. Robot 2R

- Define the architecture of the model (Feedforward Neural Network) to learn forward kinematics.




In [98]:
import torch.nn as nn
import torch
import torch.optim as optim

class ForwardKinematicsModel(nn.Module):
    def __init__(self, hidden_size=hidden_size, dropout_rate=0.3):
        super(ForwardKinematicsModel, self).__init__()
        # Define a feedforward network with configurable hidden_size and dropout
        self.fc1 = nn.Linear(2, hidden_size)  # Input layer
        self.dropout1 = nn.Dropout(p=dropout_rate)    # Dropout after first hidden layer
        self.fc2 = nn.Linear(hidden_size, hidden_size)  # Second hidden layer
        self.dropout2 = nn.Dropout(p=dropout_rate)    # Dropout after second hidden layer
        self.fc3 = nn.Linear(hidden_size, 2)  # Output layer

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)  # Apply dropout after first hidden layer
        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)  # Apply dropout after second hidden layer
        x = self.fc3(x)
        return x
model = ForwardKinematicsModel()

- Hyperparameter Search

Using Grid Search

In [93]:
# PyTorch Regressor Wrapper
class PyTorchRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, hidden_size=hidden_size  , lr=learning_rate, epochs=num_epochs):
        self.hidden_size = hidden_size
        self.lr = lr
        self.epochs = epochs
        self.model = ForwardKinematicsModel(hidden_size=self.hidden_size)
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
        self.criterion = nn.MSELoss()

    def fit(self, X, y):
        X_train = torch.tensor(X, dtype=torch.float32)
        y_train = torch.tensor(y, dtype=torch.float32)
        for epoch in range(self.epochs):
            self.model.train()
            self.optimizer.zero_grad()
            output = self.model(X_train)
            loss = self.criterion(output, y_train)
            loss.backward()
            self.optimizer.step()
        return self

    def predict(self, X):
        X_test = torch.tensor(X, dtype=torch.float32)
        self.model.eval()
        with torch.no_grad():
            predictions = self.model(X_test)
        return predictions.numpy()

# Load your dataset
df = pd.read_csv('logfiler2.csv', sep=';')
X = df[['j0', 'j1']].values
y = df[['ft_x', 'ft_y']].values

# Split dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Set up hyperparameters to search
param_grid = {
    'hidden_size': [32, 64, 128],  # Try different numbers of hidden units
    'lr': [0.001, 0.01],           # Try different learning rates
    'epochs': [100, 200],          # Try different numbers of epochs
}

# Instantiate the model wrapper
pytorch_model = PyTorchRegressor()

# Use GridSearchCV
grid_search = GridSearchCV(estimator=pytorch_model, param_grid=param_grid, cv=3, verbose=2, n_jobs=1)
grid_search.fit(X_train, y_train)

# Get the best two combinations of hyperparameters
results = pd.DataFrame(grid_search.cv_results_)
results = results.sort_values(by='mean_test_score', ascending=False)

# Print the best and second-best hyperparameters
print("Best combination of hyperparameters:")
print(results.iloc[0][['param_hidden_size', 'param_lr', 'param_epochs', 'mean_test_score']])
print("\nSecond-best combination of hyperparameters:")
print(results.iloc[1][['param_hidden_size', 'param_lr', 'param_epochs', 'mean_test_score']])

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
test_loss = mean_squared_error(y_test, y_pred)
print(f"\nTest MSE of the best model: {test_loss:.4f}")

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] END ...............epochs=100, hidden_size=32, lr=0.001; total time=   0.0s
[CV] END ...............epochs=100, hidden_size=32, lr=0.001; total time=   0.0s
[CV] END ...............epochs=100, hidden_size=32, lr=0.001; total time=   0.0s
[CV] END ................epochs=100, hidden_size=32, lr=0.01; total time=   0.0s
[CV] END ................epochs=100, hidden_size=32, lr=0.01; total time=   0.0s
[CV] END ................epochs=100, hidden_size=32, lr=0.01; total time=   0.0s
[CV] END ...............epochs=100, hidden_size=64, lr=0.001; total time=   0.0s
[CV] END ...............epochs=100, hidden_size=64, lr=0.001; total time=   0.0s
[CV] END ...............epochs=100, hidden_size=64, lr=0.001; total time=   0.0s
[CV] END ................epochs=100, hidden_size=64, lr=0.01; total time=   0.0s
[CV] END ................epochs=100, hidden_size=64, lr=0.01; total time=   0.0s
[CV] END ................epochs=100, hidden_size

ValueError: 
All the 36 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
24 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "<ipython-input-93-1abad65d442c>", line 17, in fit
    output = self.model(X_train)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1736, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1747, in _call_impl
    return forward_call(*args, **kwargs)
  File "<ipython-input-92-b4ecd9874a06>", line 16, in forward
    x = torch.relu(self.fc1(x)

- Train the models on joint angle inputs to predict fingertip positions.

In [99]:
# Hyperparameters
hidden_size = 32
learning_rate = 0.001
num_epochs = 200

- Define the loss function and optimizer

In [100]:
# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [101]:
# Convert to PyTorch tensors only if necessary
X_train = X_train.clone().detach().float() if isinstance(X_train, torch.Tensor) else torch.tensor(X_train, dtype=torch.float32)
y_train = y_train.clone().detach().float() if isinstance(y_train, torch.Tensor) else torch.tensor(y_train, dtype=torch.float32)
X_val = X_val.clone().detach().float() if isinstance(X_val, torch.Tensor) else torch.tensor(X_val, dtype=torch.float32)
y_val = y_val.clone().detach().float() if isinstance(y_val, torch.Tensor) else torch.tensor(y_val, dtype=torch.float32)
X_test = X_test.clone().detach().float() if isinstance(X_test, torch.Tensor) else torch.tensor(X_test, dtype=torch.float32)
y_test = y_test.clone().detach().float() if isinstance(y_test, torch.Tensor) else torch.tensor(y_test, dtype=torch.float32)

In [102]:
# Initialize variables for early stopping
best_val_loss = float('inf')
patience = 3
no_improvement_epochs = 0

# Training loop with early stopping
for epoch in range(num_epochs):
    model.train()

    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Evaluate on validation set
    if (epoch + 1) % 10 == 0:
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val)
            val_loss = criterion(val_outputs, y_val)

            # Calculate additional metrics
            mae = mean_absolute_error(y_val.numpy(), val_outputs.numpy())
            r2 = r2_score(y_val.numpy(), val_outputs.numpy())

            print(f"Epoch [{epoch+1}/{num_epochs}], "
                  f"Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}, "
                  f"MAE: {mae:.4f}, R^2: {r2:.4f}")

            # Early stopping check
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                no_improvement_epochs = 0
            else:
                no_improvement_epochs += 1

            if no_improvement_epochs >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}. Best Val Loss: {best_val_loss:.4f}")
                break

        model.train()  # Switch back to training mode

Epoch [10/200], Train Loss: 0.0423, Val Loss: 0.0150, MAE: 0.0990, R^2: -0.8441
Epoch [20/200], Train Loss: 0.0293, Val Loss: 0.0135, MAE: 0.0941, R^2: -0.6527
Epoch [30/200], Train Loss: 0.0219, Val Loss: 0.0127, MAE: 0.0914, R^2: -0.5391
Epoch [40/200], Train Loss: 0.0174, Val Loss: 0.0116, MAE: 0.0868, R^2: -0.4016
Epoch [50/200], Train Loss: 0.0147, Val Loss: 0.0107, MAE: 0.0833, R^2: -0.3005
Epoch [60/200], Train Loss: 0.0127, Val Loss: 0.0100, MAE: 0.0807, R^2: -0.2173
Epoch [70/200], Train Loss: 0.0113, Val Loss: 0.0094, MAE: 0.0781, R^2: -0.1359
Epoch [80/200], Train Loss: 0.0102, Val Loss: 0.0088, MAE: 0.0755, R^2: -0.0622
Epoch [90/200], Train Loss: 0.0094, Val Loss: 0.0082, MAE: 0.0728, R^2: 0.0038
Epoch [100/200], Train Loss: 0.0087, Val Loss: 0.0078, MAE: 0.0705, R^2: 0.0548
Epoch [110/200], Train Loss: 0.0082, Val Loss: 0.0075, MAE: 0.0686, R^2: 0.0939
Epoch [120/200], Train Loss: 0.0077, Val Loss: 0.0072, MAE: 0.0670, R^2: 0.1246
Epoch [130/200], Train Loss: 0.0074, Val 

In [103]:
# Test the model
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    test_loss = criterion(test_outputs, y_test)
    mae_test = mean_absolute_error(y_test.numpy(), test_outputs.numpy())
    r2_test = r2_score(y_test.numpy(), test_outputs.numpy())

    print(f"Test Loss: {test_loss.item():.4f}")
    print(f"Test MAE: {mae_test:.4f}")
    print(f"Test R^2: {r2_test:.4f}")


Test Loss: 0.0041
Test MAE: 0.0505
Test R^2: 0.5014


##3. Compare Jacobians##



3.1. Compute the Jacobian matrix for the learned forward kinematics using automatic differentiation.



In [134]:
# Load your data (replace this with your actual dataset loading method)
df = pd.read_csv('logfiler2.csv', sep=';')
X = df[['j0', 'j1']].values  # Joint angles
y = df[['ft_x', 'ft_y']].values  # Fingertip positions

# Convert to torch tensor
X_test = torch.tensor(X, dtype=torch.float32)

In [135]:


# Assuming X_test is a tensor and you want to extract the first sample
X_test_sample = X_test[0].clone().detach().unsqueeze(0).float()  # Add batch dimension and detach
print(X_test_sample)

tensor([[ 0.0550, -0.0120]])


In [140]:
def FK(model, theta):
    # Reshape to batch size 1 (equivalent to tf.reshape)
    t = theta.reshape(1, 2)

    # Pass through the model
    out = model(t)

    # Reshape the output to a 1D vector (equivalent to tf.reshape)
    out = out.reshape(2,)

    return out

def FK_Jacobian(model, x):
    # Ensure x requires gradients for gradient computation
    x = x.requires_grad_(True)

    # Forward pass to get y from FK
    y = FK(model, x)

    # Initialize an empty list to store Jacobian columns
    jacobian = []

    # Compute the gradient of each component of y with respect to x
    for i in range(y.shape[0]):
        # Gradient of y[i] with respect to x
        grad_y = torch.autograd.grad(y[i], x, retain_graph=True, create_graph=True)[0]
        jacobian.append(grad_y)

    # Stack the Jacobian columns to form the Jacobian matrix
    jacobian_matrix = torch.stack(jacobian, dim=0).squeeze()

    return jacobian_matrix

3.2. Compare the computed Jacobian with the analytical Jacobian for the 2-joint robot.

In [141]:
import numpy as np

# Analytical Jacobian for a 2-DOF robot
def analytical_jacobian_from_test(data):
    # Extract joint angles from data
    j0 = data[0]  # j0
    j1 = data[1]  # j1

    l1 = 1.0  # Link 1 length
    l2 = 1.0  # Link 2 length

    # Calculate the Jacobian elements using the analytical formula
    J = np.array([
        [-l1 * np.sin(j0) - l2 * np.sin(j0 + j1), -l2 * np.sin(j0 + j1)],
        [l1 * np.cos(j0) + l2 * np.cos(j0 + j1), l2 * np.cos(j0 + j1)]
    ])
    return J

In [142]:
import numpy as np
import torch

# Numerical Jacobian (calculated by FK_Jacobian)
X_test_sample = X_test[0].unsqueeze(0).clone().detach().float()
jacobian_numerical = FK_Jacobian(model, X_test_sample)

# Analytical Jacobian (calculated using analytical formula)
X_test_data = X_test[0].numpy()  # Extract joint angles for comparison
J_analytical = analytical_jacobian_from_test(X_test_data)

# Print both Jacobians
print("Numerical Jacobian (PyTorch):\n", jacobian_numerical)
print("Analytical Jacobian (2-DOF Robot):\n", J_analytical)

# Compare both Jacobians by calculating the difference
jacobian_difference = (jacobian_numerical.detach().numpy() - J_analytical)
print("Difference between Numerical and Analytical Jacobians:\n", jacobian_difference)

Numerical Jacobian (PyTorch):
 tensor([[-0.0023,  0.0118],
        [ 0.0407,  0.0241]], grad_fn=<SqueezeBackward0>)
Analytical Jacobian (2-DOF Robot):
 [[-0.09795902 -0.04298675]
 [ 1.99756354  0.99907565]]
Difference between Numerical and Analytical Jacobians:
 [[ 0.09568931  0.05483213]
 [-1.95689448 -0.97500462]]
